In [10]:
import pandas as pd
from time import time
import pyarrow.parquet as pq
from sqlalchemy import create_engine

In [11]:
# Fichier Parquet distant
#file_path = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet"
file_path = "/data/yellow_tripdata_2021-01.parquet"
parquet_file = pq.ParquetFile(file_path)

In [12]:
# Connexion à la base PostgreSQL
engine = create_engine("postgresql://root:root@db:5432/ny_taxi")

# Lire un premier batch pour créer le schéma
first_batch = next(parquet_file.iter_batches(batch_size=100_000))
df = first_batch.to_pandas()

In [13]:
query = """
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
"""
pd.read_sql(query, con=engine)


,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,yellow_taxi_data,root,None,True,False,False,False


In [14]:
# Traitement des dates
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Créer la table avec le schéma
df.head(0).to_sql(name='yellow_taxi_data', con=engine, if_exists='replace')

# Insérer le premier batch
df.to_sql(name='yellow_taxi_data', con=engine, if_exists='append')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5,NaN
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,NaN
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,NaN
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,NaN
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2021-01-04 14:04:31,2021-01-04 14:08:52,3.0,0.70,1.0,N,234,224,2,5.0,2.5,0.5,0.00,0.0,0.3,8.30,2.5,NaN
99996,1,2021-01-04 14:18:46,2021-01-04 14:35:45,2.0,3.30,1.0,N,234,236,1,14.5,2.5,0.5,3.55,0.0,0.3,21.35,2.5,NaN
99997,1,2021-01-04 14:42:41,2021-01-04 14:59:22,2.0,4.70,1.0,N,236,79,1,17.0,2.5,0.5,4.05,0.0,0.3,24.35,2.5,NaN
99998,2,2021-01-04 14:39:02,2021-01-04 15:09:37,2.0,17.95,2.0,N,132,148,1,52.0,0.0,0.5,5.00,0.0,0.3,60.30,2.5,NaN


In [15]:
# Boucler sur les batches suivants
for batch in parquet_file.iter_batches(batch_size=100_000):
    t_start = time()
    df = batch.to_pandas()
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
    df.to_sql(name='yellow_taxi_data', con=engine, if_exists='append')
    t_end = time()
    print("inserted another chunk ..., took %.3f second" %(t_end - t_start))

inserted another chunk ..., took 16.096 second
inserted another chunk ..., took 16.531 second
inserted another chunk ..., took 19.429 second
inserted another chunk ..., took 38.122 second
inserted another chunk ..., took 45.863 second
inserted another chunk ..., took 52.079 second
inserted another chunk ..., took 47.041 second
inserted another chunk ..., took 19.972 second


PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)